# Imports

In [3]:
import pandas as pd 
import numpy as np
#pyspark for analysis:
from pyspark.sql import SparkSession
import tensorflow as tf

In [4]:
spark = SparkSession.builder.appName("Py-SQL analysis").getOrCreate()

24/11/16 19:21:49 WARN Utils: Your hostname, Admins-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.5.82 instead (on interface en1)
24/11/16 19:21:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 19:21:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
